In [1]:
import os
import sys
import re
import time
import random
import warnings
import collections
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

sys.path.append('../../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

# Milliman PAC File ETL
1) log into SFTP site: https://secureftp.milliman.com  
2) In the "from milliman" folder, go to the latest directory  
     - In this case, the latest directory is "20221216"  
3) Make sure you have access to the following folder: \\Main Street Health\MSH_Strategy_Group - ACO Analyses\Raw MIlliman Flat FIles\  
     - create folder with same datestamp as file from milliman, in this case "20221216"  
4) Download the files from the SFTP site  
     - in this case there are 3  
5) copy the data disctionary to the folder on the network drive.  
6) download the prospective and retrospective CSV files into database tables in both the MSH and CB datagrip environments (Alan has a place he puts them)  
7) delete the last 30 columns from these database tables (we will create a "pivot file" with these 30 columns):  
8) with these 30 columns, run Alan's "pivot code" to create these files:  
     - strategic_milliman_prosp_20221128_pivot
     -  strategic_milliman_retro_20221128_pivot
9) copy 4 files to the "20221216" folder  
     - prospective flat file (without the 30 MA columns)
     - retrospective flat file (without the 30 MA columns)
     - prospective MA pivot file
     - retrospective MA pivor file
10) email Ali and Ben that process is complete with the following summaries that you can pull via queries and paste in excel into the same folder.  

### Config

In [2]:
data_dir = '/Users/bp/workspace/msh/milliman_data'
current_folder = '20250221'
file_name = 'MSSP Explorer v2026.2 - Regional Risk Scores.txt'

### File load

In [3]:
df = pd.read_csv(f'{data_dir}/{current_folder}/{file_name}',  encoding='latin1', low_memory=False, delimiter='\t')
df = cb_utils.df_format_columns(df)
print(f'{df.shape[0]} rows, {df.shape[1]} columns')
df.head()

551480 rows, 23 columns


,id,year,assignment,py_esrd,py_disa,py_agdu,py_agnd,ret_v24_rs_esrd,ret_v24_rs_disa,ret_v24_rs_agdu,ret_v24_rs_agnd,pro_v24_rs_esrd,pro_v24_rs_disa,pro_v24_rs_agdu,pro_v24_rs_agnd,ret_v28_rs_esrd,ret_v28_rs_disa,ret_v28_rs_agdu,ret_v28_rs_agnd,pro_v28_rs_esrd,pro_v28_rs_disa,pro_v28_rs_agdu,pro_v28_rs_agnd
0,896D70502C59,2023,Prospective,48.50,267.25,174.92,3054.75,45.563765,264.799867,185.352816,2913.766841,46.999190,276.231623,188.441432,3040.660350,45.563765,267.800541,188.572411,2971.281976,46.999190,277.737998,190.715628,3086.630528
1,37F1523492C5,2023,Prospective,10.42,116.75,69.83,1628.17,10.052352,116.597915,73.756190,1565.301588,10.361836,121.225978,75.094883,1634.400614,10.052352,117.450100,75.007867,1592.328261,10.361836,121.267115,76.020340,1655.136226
2,ECA5013CECFE,2023,Prospective,1.17,40.50,12.92,1162.58,1.108979,39.010785,13.892329,1114.515475,1.146193,40.706525,14.112510,1164.110217,1.108979,39.544563,14.138896,1134.354477,1.146193,40.975394,14.279395,1179.414379
3,A07182128E94,2023,Prospective,2.83,645.42,298.92,19.92,2.725681,660.135712,316.719279,20.971376,2.791831,678.430564,323.483906,21.702128,2.725681,658.759458,316.981922,20.526905,2.791831,672.530916,322.613528,21.152068
4,673989,2023,Prospective,3.00,39.42,32.92,307.83,2.922739,36.645809,32.469966,287.074735,3.030622,38.272071,33.272137,297.578203,2.922739,37.077784,32.729429,296.171319,3.030622,38.414559,33.356732,305.639759


In [4]:
conn = cb_utils.get_engine(source='msh_analytics')
pro_table_name = f'milliman_regional_risk_scores_{current_folder}'
pro_schema = 'raw'
print(f'select * from {pro_schema}.{pro_table_name}')
# pro_df.to_sql(pro_table_name, conn, schema=pro_schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

select * from raw.milliman_regional_risk_scores_20250221


In [5]:
df.to_sql(pro_table_name, conn, schema=pro_schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

551480